In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline

import transform

import pymap3d.vincenty as pmv
from skopt import gp_minimize
from skopt.space import Real, Integer

import warnings
warnings.filterwarnings('ignore')

#BASE_DIR = '/Users/vadimzubkov/Desktop/smartphone-decimeter-2022/train/2021-03-16-US-MTV-1'
#BASE_DIR = Path('/Users/vadimzubkov/Desktop/smartphone-decimeter-2022/train/2021-03-16-US-MTV-1')

path = '/Users/vadimzubkov/Desktop/smartphone-decimeter-2022'

In [2]:
def bias_correction_phone(args):
        phone, phone_df = args

        B = np.deg2rad(phone_df['LatitudeDegrees'].values)
        L = np.deg2rad(phone_df['LongitudeDegrees'].values)
        H = np.zeros_like(B)
        BLH = transform.BLH(lat=B, lng=L, hgt=H) #
        J = transform.jacobian_BL_to_EN(BLH) #

        t_ref  = phone_df['UnixTimeMillis'].min()
        TIME   = 1e-3 * (phone_df['UnixTimeMillis'] - t_ref).values
        dotB   = InterpolatedUnivariateSpline(TIME, B, k=3).derivative()(TIME)
        dotL   = InterpolatedUnivariateSpline(TIME, L, k=3).derivative()(TIME)
        dotBL  = np.stack([dotB, dotL], axis=1)
        dotEN  = np.einsum('nij,nj->ni', J, dotBL)
        absV   = np.sqrt(np.sum(dotEN**2, axis=1))
        th_az  = np.arctan2(dotEN[:, 0], dotEN[:, 1])

        cos_az = np.cos(th_az)
        sin_az = np.sin(th_az)
        valid  = (absV > (5 / 3.6))
        cos_az = InterpolatedUnivariateSpline(TIME[valid], cos_az[valid], k=1, ext=3)(TIME)
        sin_az = InterpolatedUnivariateSpline(TIME[valid], sin_az[valid], k=1, ext=3)(TIME)
        th_az  = np.arctan2(sin_az, cos_az)
        cos_az = np.cos(th_az)
        sin_az = np.sin(th_az)

        delta_X  = - BIAS_X
        delta_Y  = - BIAS_Y
        delta_E  = (  cos_az * delta_X) + (sin_az * delta_Y)
        delta_N  = (- sin_az * delta_X) + (cos_az * delta_Y)
        delta_EN = np.stack([delta_E, delta_N], axis=0) # shape = (2, N)
        Jinv = np.linalg.inv(np.mean(J, axis=0))
        delta_BL_rad = Jinv @ delta_EN
        delta_BL_deg = np.rad2deg(delta_BL_rad)

        output_df = pd.DataFrame({
            'tripId'               : phone_df['tripId'],
            'UnixTimeMillis' : phone_df['UnixTimeMillis'],
            'LatitudeDegrees'              : phone_df['LatitudeDegrees'] + delta_BL_deg[0, :],
            'LongitudeDegrees'              : phone_df['LongitudeDegrees'] + delta_BL_deg[1, :],
        })
        return output_df
    
def bias_correction(base_df):
        output_df = base_df.sort_values(['tripId', 'UnixTimeMillis']).reset_index(drop=True).copy()
        output_df_list = map(bias_correction_phone, base_df.groupby('tripId'))
        _df = pd.concat(output_df_list, axis=0)
        _df = _df.sort_values(['tripId', 'UnixTimeMillis']).reset_index(drop=True)
        output_df[['LatitudeDegrees','LongitudeDegrees']] = _df[['LatitudeDegrees','LongitudeDegrees']]
        return output_df

In [3]:
# Compute distance by Vincenty's formulae
def vincenty_distance(llh1, llh2):
    """
    Args:
        llh1 : [latitude,longitude] (deg)
        llh2 : [latitude,longitude] (deg)
    Returns:
        d : distance between llh1 and llh2 (m)
    """
    d, az = np.array(pmv.vdist(llh1[:, 0], llh1[:, 1], llh2[:, 0], llh2[:, 1]))

    return d


# Compute score
def calc_score(llh, llh_gt):
    """
    Args:
        llh : [latitude,longitude] (deg)
        llh_gt : [latitude,longitude] (deg)
    Returns:
        score : (m)
    """
    d = vincenty_distance(llh, llh_gt)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])

    return score

In [4]:
BIAS_X = 0.3
BIAS_Y = 0.4

train_base = pd.read_csv('baseline_train.csv')
ground_base = pd.read_csv('train_gt.csv')
llh_gt = ground_base[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
llh_bl = train_base[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
    
score_bl = calc_score(llh_bl, llh_gt)
print(f'score without bias correction: {score_bl}')

train = bias_correction(train_base)
llh_bias = train[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
score_bias = calc_score(llh_bias, llh_gt)
print(f'score with bias correction: {score_bias}')

score without bias correction: 3.3750655588077088
score with bias correction: 3.310785308742261


In [5]:
test_base_rtk = pd.read_csv('sub_savgol.csv') #submit_0722
test_base_wls = pd.read_csv('best_public.csv')
test_rtk = bias_correction(test_base_rtk)
test_wls = bias_correction(test_base_wls)

In [6]:
test_rtk.to_csv('sub_savgol_bias.csv', index=False)
test_wls.to_csv('sub_public_bias.csv', index=False)